# Am I the Asshole Data Loading
This file will be dedicated to explore how to craft responses to posts in the subreddit r/AmITheAsshole. 

In [1]:
import os 
import sys
import importlib
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

In [2]:
sys.path.append("..")

# load enviornment variables for praw to work later
load_dotenv(dotenv_path=Path("../settings.env"))

True

In [3]:
client = bigquery.Client()

In [4]:
# helper function to check if table exists
def gcp_table_exists(client: bigquery.Client, table_id: str):
    try:
        client.get_table(table_id)  # Make an API request.
        return True
    except NotFound:
        return False

## Create Dataset
Using Google Bigquery, create datasets for the data that is about to be loaded. The dataset will be called **AITA_dataset**

In [5]:
PROJ_NAME = "bonion"
DATASET_NAME = "AITA_dataset"

#### Get all dataset ID's
To prevent overwriting pre-existing datasets, get all dataset id's to check before any creation step.

In [6]:
# Get all datasets 
datasets = list(client.list_datasets())  # Make an API request.
dataset_ids = [dataset.dataset_id for dataset in datasets]
project = client.project

if datasets:
    print("Datasets in project {}:".format(project))
    for dataset in datasets:
        print("\t{}".format(dataset.dataset_id))
else:
    print("{} project does not contain any datasets.".format(project))

Datasets in project bonion:
	AITA_dataset


In [7]:
# create dataset
if DATASET_NAME not in dataset_ids:
    dataset = bigquery.Dataset("{}.{}".format(PROJ_NAME, DATASET_NAME))
    dataset.location = "US"

    # send dataset to API for completion
    initial_post_dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

## Post Table
The initial post for each of the comments will need to be stored to reference its content which dictated the reply.

In [8]:
# table id definitions
post_table_id = "{}.{}.post_table".format(PROJ_NAME, DATASET_NAME)
post_reply_table_id = "{}.{}.post_reply_table".format(PROJ_NAME, DATASET_NAME)
post_reply_top_children_table_id = "{}.{}.post_reply_top_children".format(PROJ_NAME, DATASET_NAME)

#### Get all Table ID's
To prevent overwriting pre-existing tables, get all table id's to check before any creation step.

In [9]:
tables = list(client.list_tables(DATASET_NAME))  # Make an API request.
table_ids = [table.table_id for table in tables]

print("Tables contained in '{}':".format(DATASET_NAME))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Tables contained in 'AITA_dataset':
bonion.AITA_dataset.post_reply_table
bonion.AITA_dataset.post_reply_top_children
bonion.AITA_dataset.post_table


In [10]:
post_table_schema = [
    bigquery.SchemaField("reddit_post_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("post_title", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("post_self_text", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("upvotes", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("num_responses", "INTEGER", mode="REQUIRED")
]

if not gcp_table_exists(client, table_id=post_table_id): 
    post_table = bigquery.Table(post_table_id, schema=post_table_schema)
    post_table.description = """
        A table which holds popular posts from the subreddit r/AITA
    """

    table = client.create_table(post_table)
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

## Post Reply Table
This table holds all of the responses for each of the posts in the post table.

In [11]:
post_reply_table_schema = [
    bigquery.SchemaField("reddit_post_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("comment_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("comment_contents", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("upvotes", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("num_responses", "INTEGER", mode="REQUIRED")
]

if not gcp_table_exists(client, table_id=post_reply_table_id): 
    post_reply_table = bigquery.Table(post_reply_table_id, schema=post_reply_table_schema)
    post_reply_table.description = """
        A table which holds the most popular replys to saved posts from the subreddit r/AITA
    """

    table = client.create_table(post_reply_table)
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

## Post Reply Top Children
This table will hold replys to the initial post reply. This will be used for also dictating the performance of a reply. Because there can be infinite replies to any given post or reply, each post reply id will be limited to some number of replies. These replies will likley be based off of the same metrics as the original post for "quality". 

In [12]:
post_reply_top_children_table_schema = [
    bigquery.SchemaField("parent_comment_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("comment_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("comment_contents", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("upvotes", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("num_responses", "INTEGER", mode="REQUIRED")
]   

if not gcp_table_exists(client, table_id=post_reply_top_children_table_id): 
    post_reply_top_children_table = bigquery.Table(post_reply_top_children_table_id, schema=post_reply_top_children_table_schema)
    post_reply_top_children_table.description = """
        This table will hold replys to the initial post reply. 
        This will be used for also dictating the performance of a reply. 
        Because there can be infinite replies to any given post or reply, 
        each post reply id will be limited to some number of replies. 
        These replies will likley be based off of the same metrics as 
        the original post for "quality". 
    """

    table = client.create_table(post_reply_top_children_table)
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

## Collect Data
Using my PrawInstance Object, I will be collecting the top posts from the subreddit. I will be storing this data in google bigquery.

In [20]:
import praw_instance
importlib.reload(praw_instance)

<module 'praw_instance' from '/home/cstainsby/class/dataProj/bonion/src/notebooks/../praw_instance.py'>

In [14]:
subreddit_name = "amitheasshole"

In [15]:
praw_inst = praw_instance.PrawInstance()

### Store Top 1000 Posts

In [16]:
top_10000_posts = praw_instance.get_top_by_subreddit(praw_inst, subreddit_name, limit=1000)

KeyboardInterrupt: 

In [ ]:
# store the top 1000 posts into gcp
# convert data into json rows 
post_table_df = praw_instance.post_dict_to_df(top_10000_posts)

post_table_df.head(10)

In [ ]:
# cast table columns to type
post_table_df["reddit_post_id"] = post_table_df["reddit_post_id"].astype(str)
post_table_df["post_title"] = post_table_df["post_title"].astype(str)
post_table_df["post_self_text"] = post_table_df["post_self_text"].astype(str)
post_table_df["upvotes"] = post_table_df["upvotes"].astype(int)
post_table_df["num_responses"] = post_table_df["num_responses"].astype(int)
post_table_df.info()

#### Push Data

In [ ]:
import pandas_gbq

In [ ]:
pandas_gbq.to_gbq(post_table_df, post_table_id, project_id=PROJ_NAME)

### Get Top Comments and Replies for the top posts

In [17]:
# get the top post ids

query = """
    SELECT reddit_post_id
    FROM {}
""".format(post_table_id)

post_table_ids_df = pd.read_gbq(query, project_id=PROJ_NAME)

post_table_ids_df.head(4)

,reddit_post_id
0,jyk2ac
1,ukzctc
2,u90414
3,socrlh


In [23]:
comment_and_reply_dict_list = []

# I will batch the groups of comments I'm grabbing
# so far I have pulled indices 0 thru 20
start_index = 0 
stop_index = 20

for index, row in post_table_ids_df.iterrows():
    if index >= start_index and index <= stop_index: 
        print("On index {} of {}".format(index, stop_index - start_index))
        comment_and_reply_dict = praw_instance.get_top_comments_and_top_replies_by_post_id(
            praw_inst,
            row["reddit_post_id"],
            comment_limit=10,
            reply_limit=5
        )

        comment_and_reply_dict_list.append(comment_and_reply_dict)

On index 0 of 998
On index 1 of 998
On index 2 of 998
On index 3 of 998
On index 4 of 998
On index 5 of 998
On index 6 of 998
On index 7 of 998
On index 8 of 998
On index 9 of 998
On index 10 of 998
On index 11 of 998
On index 12 of 998
On index 13 of 998
On index 14 of 998
On index 15 of 998
On index 16 of 998
On index 17 of 998
On index 18 of 998
On index 19 of 998
On index 20 of 998


In [ ]:
# convert the data to a dataframe

for post_i, comment_and_replies_dict in enumerate(comment_and_reply_dict_list):
    comment_and_replies_df_at_post_i =  praw_instance.comment_and_reply_dict_to_df(comment_and_replies_dict)
    
